In [38]:
import cv2
import os
import numpy as np

# Load templates from a directory
template_dir = 'templates/'
templates = []
template_names = []


def load_templates(rf = 0.3):
    #load templates
    for template_file in os.listdir(template_dir):
        template_path = os.path.join(template_dir, template_file)
        template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)  # Read the template in grayscale
        template = cv2.resize(template, None, fx=rf, fy=rf)
        template = cv2.flip(template, 1)
        templates.append(template)
        template_names.append(os.path.splitext(template_file)[0])

def draw_ROI(background, original_image, original_frame, prev_cX):
    #difference image
    diff = cv2.absdiff(background, original_image)
    #thresholded image
    _, thresholded_diff = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    #contours
    contours, _ = cv2.findContours(thresholded_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ismoving = False
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        # area = cv2.contourArea(largest_contour)
        # if area<1000:
        #     cv2.putText(original_frame, f'No Hand Detected', (10, 30), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
        #     continue
        x, y, w, h = cv2.boundingRect(largest_contour)
        moments = cv2.moments(largest_contour)

        cv2.rectangle(original_frame, (original_frame.shape[1]-300 + x, y), (original_frame.shape[1]-300 + x + w, y + h), (255,0,0), 2)            
        if moments["m00"]:
            centroid_x = int(moments["m10"] / moments["m00"])
            centroid_y = int(moments["m01"] / moments["m00"])
                        # Check if the centroid has moved a lot horizontally
            if abs(centroid_x - prev_cX) > 15 and moments["m00"] != 0:
                prev_cX= centroid_x
                ismoving =  True

            original_frame = cv2.circle(original_frame, (original_frame.shape[1]-300 +centroid_x, centroid_y), 5, 128, -1)
            original_frame = cv2.putText(original_frame, f'Centroid', (original_frame.shape[1]-300 +centroid_x, centroid_y), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 2)

    return original_frame, thresholded_diff, ismoving, prev_cX

def classify(original_image, original_frame, ismoving):
    # Initialize variables to keep track of the best match
    best_match_value = -1
    best_template_index = -1
    best_scale_factor = 1.0

    # Iterate through templates
    for i, template in enumerate(templates):
        # Initialize variables for the current template
        template_best_match_value = -1
        template_best_scale_factor = 1.0

        # Iterate through different scales of the image
        for scale_factor in np.linspace(0.1, 1.0, 10):
            scaled_image = cv2.resize(original_image, None, fx=scale_factor, fy=scale_factor)
            if scaled_image.shape[0]<template.shape[0] or scaled_image.shape[1]<template.shape[1]:
                continue
            result = cv2.matchTemplate(scaled_image, template, cv2.TM_CCOEFF_NORMED)
            _, max_val, _, _ = cv2.minMaxLoc(result)

            # Update the best match information for the current template
            if max_val > template_best_match_value:
                template_best_match_value = max_val
                template_best_scale_factor = scale_factor

        # Update the overall best match information
        if template_best_match_value > best_match_value:
            best_match_value = template_best_match_value
            best_template_index = i
            best_scale_factor = template_best_scale_factor

    corr_threshold = 0.74
    # Draw a rectangle around the matched template
    if best_template_index != -1 and best_match_value>corr_threshold:
        # Display the class label
        class_label = template_names[best_template_index]
        if ismoving:
            if class_label == "rock":
                cv2.putText(original_frame, f"Don't hit me with that rock!", (10, 70), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            elif class_label == "paper":
                cv2.putText(original_frame, f"Bye!", (10, 70), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)

        cv2.putText(original_frame, f'Class: {class_label}', (10, 30), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
        cv2.putText(original_frame, f'corr: {best_match_value}', (10, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
    else:
        cv2.putText(original_frame, f'No Hand Detected', (10, 30), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
    return original_frame

def main():
    load_templates()
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Couldn't open the webcam.")
        return
    ret, background = cap.read()
    prev_cX = 0
    #set background
    
    frame_count = 0
    while True:
        # Capture frame from webcam
        ret, original_frame = cap.read()
        original_frame = cv2.flip(original_frame, 1)
        frame_count += 1
        if not ret:
            print("Error: Couldn't read a frame from the webcam.")
            break
        set_background_after = 60
        if frame_count<set_background_after:
            continue
        if frame_count==set_background_after:
            background = cv2.cvtColor(original_frame, cv2.COLOR_BGR2GRAY)[:300,-300:]
             
        #Region to consider
        roi = cv2.cvtColor(original_frame, cv2.COLOR_BGR2GRAY)[:300,-300:]
        original_frame, thresholded_diff, is_moving, prev_cX = draw_ROI(background, roi,original_frame, prev_cX)
        original_frame = classify(roi, original_frame, is_moving)
        cv2.rectangle(original_frame, (original_frame.shape[1]-300, 0), (original_frame.shape[1], 300), (0,255,0), 2)
        cv2.imshow('Template Matching', original_frame)
        cv2.imshow('threshold', thresholded_diff)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q') :
            break

    # Release the webcam and close all windows
    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()
